In [1]:
from __future__ import print_function

import sys
import numpy as np
from sklearn.preprocessing import QuantileTransformer
from sklearn.model_selection import train_test_split

sys.path.append('..')
import tensorflow as tf
#from supplementary_code_direct_ranker.DirectRanker import directRanker
#from supplementary_code_direct_ranker.helpers import readData, nDCGScorer_cls, MAP_cls
import DirectRanker as drr
import helpers as hlps

Instructions for updating:
non-resource variables are not supported in the long term


# Read data

In [2]:
x_test, y_test, q_test = hlps.readData(data_path="/Users/wery/Desktop/MSLR-WEB10K/Fold1/test.txt", binary=True, at=10, number_features=136, bin_cutoff=1.5, cut_zeros=True)
x_train, y_train, q_train = hlps.readData(data_path="/Users/wery/Desktop/MSLR-WEB10K/Fold1/train.txt", binary=True, at=10, number_features=136, bin_cutoff=1.5, cut_zeros=True)

# Define Ranker

In [3]:
def lambda_cost(nn, y0):
    return tf.reduce_mean(tf.log(1+tf.exp(nn))-nn)


# Load directRanker, train, and test
dr = drr.directRanker(
    feature_activation=tf.nn.tanh,
    ranking_activation=tf.nn.tanh,
    # max_steps=10000,
    # For debugging
    #cost=lambda_cost,
    max_steps=10000,
    print_step=500,
    start_batch_size=3,
    end_batch_size=5,
    start_qids=20,
    end_qids=100,
    feature_bias=True,
    hidden_layers=[100, 50, 5]
)

# Fit

In [4]:
dr.fit(x_train, y_train, ranking=True)

/Users/wery/Desktop/DirectRankerMQ2008/DirectRanker.py:207: UserWarning: `tf.layers.dropout` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dropout` instead.
  in0 = tf.layers.dropout(inputs=self.x0,
/Users/wery/Desktop/DirectRankerMQ2008/DirectRanker.py:212: UserWarning: `tf.layers.dropout` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dropout` instead.
  in1 = tf.layers.dropout(inputs=self.x1,
/Users/wery/Desktop/DirectRankerMQ2008/DirectRanker.py:218: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  nn0 = tf.layers.dense(
/Users/wery/Desktop/DirectRankerMQ2008/DirectRanker.py:230: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  nn1 = tf.layers.dense(
/Users/wery/Desktop/DirectRankerMQ2008/DirectRanker.py:242: UserWarning: `tf.layers.dropout` is de

step: 0, value: 0.9998838901519775, samples: 3, queries: 20
step: 500, value: 0.8950247764587402, samples: 3, queries: 21
step: 1000, value: 0.8235525488853455, samples: 3, queries: 23
step: 1500, value: 0.8441898822784424, samples: 3, queries: 25
step: 2000, value: 0.8656865954399109, samples: 3, queries: 27
step: 2500, value: 0.8921211361885071, samples: 3, queries: 29
step: 3000, value: 0.8498957753181458, samples: 3, queries: 32
step: 3500, value: 0.770397961139679, samples: 3, queries: 35
step: 4000, value: 0.7753650546073914, samples: 3, queries: 38
step: 4500, value: 0.7787533402442932, samples: 3, queries: 41
step: 5000, value: 0.8912742733955383, samples: 3, queries: 44
step: 5500, value: 0.9179871678352356, samples: 3, queries: 48
step: 6000, value: 0.7852871417999268, samples: 4, queries: 52
step: 6500, value: 0.8519539833068848, samples: 4, queries: 56
step: 7000, value: 0.8347021341323853, samples: 4, queries: 61
step: 7500, value: 0.8470625281333923, samples: 4, queries: 

# Evaluate

In [5]:
hlps.nDCGScorer_cls(dr, x_test, y_test, at=10)
hlps.MAP_cls(dr, x_test, y_test)

nDCG@10: 0.4113 +- 0.2414
MAP: 0.3433 +- 0.1773


0.3432874043805785